In [9]:
import apache_beam as beam

def run():
  options = {
      'staging_location': 'gs://doit-playground/staging',
      'temp_location': 'gs://doit-playground/tmp',
      'job_name': 'test',
      'project': 'doit-playground',
      'teardown_policy': 'TEARDOWN_ALWAYS',
      'no_save_main_session': True
    }
  opts = beam.pipeline.PipelineOptions(flags=[], **options)
  p = beam.Pipeline('DirectRunner', options=opts)
  p1 = p | 'query' >> beam.io.Read(beam.io.BigQuerySource(use_standard_sql=True,query='SELECT edgeIP  FROM `doit-playground.playground.akamai_errors` LIMIT 1000')) 
  p1 = p1 | 'map' >> beam.Map(map_item) 
#   p1 = p1 | 'write' >> beam.io.Write(beam.io.WriteToText('gs://doit-playground/out/test.csv'))
  p1 = p1 | 'write' >> beam.io.Write(beam.io.BigQuerySink(table='test',dataset='test',schema='msg:string'))
  
  p.run()

def map_item(rowdict):
  return {"msg":rowdict['edgeIP']}

run()

In [5]:
%%bq query 
SELECT edgeIP 
FROM `doit-playground.playground.akamai_errors` 
LIMIT 1000

edgeIP
184.28.17.42
23.52.143.46
23.67.53.100
66.61.169.84
184.27.45.165
23.212.50.113
23.212.50.113
23.67.243.154
23.73.181.201
40.135.67.148
